In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import time


In [3]:
df = pd.read_parquet('/home/samuelw/Projetos/tedas/data/raw/mimic_iv_ext_onto_sepse_rotulado.parquet')
df.head()

,subject_id,hadm_id,stay_id,charttime,fc,pas,pad,pam,fr,spo,tem,cre,lac,leu,pla,ph,uri,hem,ida,pes,alt,sexo,tem_sepse
0,10000032,29079034,39553978,2180-07-23 14:11:00,NaN,84.0,48.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
1,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
2,10000032,29079034,39553978,2180-07-23 14:13:00,NaN,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
3,10000032,29079034,39553978,2180-07-23 14:30:00,93.0,95.0,59.0,67.0,21.0,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
4,10000032,29079034,39553978,2180-07-23 15:00:00,94.0,88.0,56.0,64.0,23.0,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0


In [4]:
# Início da medição de tempo
start_time = time.time()

# Detecta as duplicatas
duplicatas = df.duplicated()

# Exibe as duplicatas encontradas
print("Linhas duplicadas:")
print(df[duplicatas])

# Remove as duplicatas (mantém a primeira ocorrência)
df = df.drop_duplicates()

# Fim da medição de tempo
end_time = time.time()

# Exibe tempo de execução
print("-" * 100)
elapsed_time = end_time - start_time
print(f"Tempo de execução: {elapsed_time:.2f} segundos")
print("-" * 100)


Linhas duplicadas:
Empty DataFrame
Columns: [subject_id, hadm_id, stay_id, charttime, fc, pas, pad, pam, fr, spo, tem, cre, lac, leu, pla, ph, uri, hem, ida, pes, alt, sexo, tem_sepse]
Index: []
----------------------------------------------------------------------------------------------------
Tempo de execução: 7.46 segundos
----------------------------------------------------------------------------------------------------


In [11]:
def selecao_por_colonia_de_formigas(
    X, y,
    n_formigas=15,
    n_iteracoes=12,
    alfa=1,
    beta=1,
    taxa_evaporacao=0.3,
    min_caracteristicas_por_formiga=5,
    max_caracteristicas_por_formiga=8
):
    n_caracteristicas = X.shape[1]
    feromonio = np.ones(n_caracteristicas)

    # Heurística baseada na AUC ROC de cada feature individual
    heuristica = np.zeros(n_caracteristicas)
    for i in range(n_caracteristicas):
        try:
            heuristica[i] = roc_auc_score(y, X[:, i])
        except:
            heuristica[i] = 0.5  # AUC neutra para evitar erros
    heuristica += 1e-6  # evitar zeros

    def avaliar(indices):
        if len(indices) == 0:
            return 0
        X_sel = X[:, indices]
        modelo = LogisticRegression(solver='liblinear', class_weight='balanced', max_iter=1000)
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scores = cross_val_score(modelo, X_sel, y, cv=kfold, scoring='roc_auc')
        return scores.mean()

    melhor_indice_caracteristicas = []
    melhor_pontuacao = -np.inf

    for iteracao in range(n_iteracoes):
        solucoes_iteracao = []
        pontuacoes_iteracao = []

        print(f"Iteração {iteracao + 1}/{n_iteracoes}")

        for _ in range(n_formigas):
            # Probabilidades combinando feromônio e heurística (AUC)
            probabilidades = (feromonio ** alfa) * (heuristica ** beta)
            probabilidades /= probabilidades.sum()

            tamanho = np.random.randint(
                min_caracteristicas_por_formiga,
                min(max_caracteristicas_por_formiga, n_caracteristicas) + 1
            )

            indices = np.random.choice(
                range(n_caracteristicas),
                size=tamanho,
                replace=False,
                p=probabilidades
            )

            pontuacao = avaliar(indices)
            solucoes_iteracao.append(indices)
            pontuacoes_iteracao.append(pontuacao)

            if pontuacao > melhor_pontuacao:
                melhor_pontuacao = pontuacao
                melhor_indice_caracteristicas = indices

        # Evaporação
        feromonio *= (1 - taxa_evaporacao)
        # Reforço
        for solucao, pontuacao in zip(solucoes_iteracao, pontuacoes_iteracao):
            for i in solucao:
                feromonio[i] += pontuacao

    return melhor_indice_caracteristicas, melhor_pontuacao

# ----------------------------
# PREPARAÇÃO DOS DADOS E EXECUÇÃO
# ----------------------------
colunas_entrada = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre',
                   'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida']
coluna_alvo = 'tem_sepse'

dados_entrada = df[colunas_entrada].copy()
dados_entrada = dados_entrada.apply(pd.to_numeric, errors='coerce')
dados_entrada.fillna(dados_entrada.mean(), inplace=True)

X = dados_entrada.values.astype(float)
y = df[coluna_alvo].values
if y.dtype == 'object' or str(y.dtype).startswith("category"):
    y = LabelEncoder().fit_transform(y)

melhores_indices, melhor_pontuacao = selecao_por_colonia_de_formigas(X, y)
melhores_nomes = [colunas_entrada[i] for i in melhores_indices]

print("\nMelhores características selecionadas:", melhores_nomes)
print("Melhor AUC (curva ROC):", melhor_pontuacao)


Iteração 1/12
Iteração 2/12
Iteração 3/12
Iteração 4/12
Iteração 5/12
Iteração 6/12
Iteração 7/12
Iteração 8/12
Iteração 9/12
Iteração 10/12
Iteração 11/12
Iteração 12/12

Melhores características selecionadas: ['spo', 'cre', 'fc', 'pas', 'pam', 'fr', 'leu', 'uri']
Melhor AUC (curva ROC): 0.6203749268796783


#### Aumentando a quantidade de iterações para 15 e o número máximo de formigas para 20

In [4]:
def selecao_por_colonia_de_formigas(
    X, y,
    n_formigas=20,
    n_iteracoes=15,
    alfa=1,
    beta=1,
    taxa_evaporacao=0.3,
    min_caracteristicas_por_formiga=5,
    max_caracteristicas_por_formiga=8
):
    n_caracteristicas = X.shape[1]
    feromonio = np.ones(n_caracteristicas)

    # Heurística baseada na AUC ROC de cada feature individual
    heuristica = np.zeros(n_caracteristicas)
    for i in range(n_caracteristicas):
        try:
            heuristica[i] = roc_auc_score(y, X[:, i])
        except:
            heuristica[i] = 0.5  # AUC neutra para evitar erros
    heuristica += 1e-6  # evitar zeros

    def avaliar(indices):
        if len(indices) == 0:
            return 0
        X_sel = X[:, indices]
        modelo = LogisticRegression(solver='liblinear', class_weight='balanced', max_iter=1000)
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scores = cross_val_score(modelo, X_sel, y, cv=kfold, scoring='roc_auc')
        return scores.mean()

    melhor_indice_caracteristicas = []
    melhor_pontuacao = -np.inf

    for iteracao in range(n_iteracoes):
        solucoes_iteracao = []
        pontuacoes_iteracao = []

        print(f"Iteração {iteracao + 1}/{n_iteracoes}")

        for _ in range(n_formigas):
            # Probabilidades combinando feromônio e heurística (AUC)
            probabilidades = (feromonio ** alfa) * (heuristica ** beta)
            probabilidades /= probabilidades.sum()

            tamanho = np.random.randint(
                min_caracteristicas_por_formiga,
                min(max_caracteristicas_por_formiga, n_caracteristicas) + 1
            )

            indices = np.random.choice(
                range(n_caracteristicas),
                size=tamanho,
                replace=False,
                p=probabilidades
            )

            pontuacao = avaliar(indices)
            solucoes_iteracao.append(indices)
            pontuacoes_iteracao.append(pontuacao)

            if pontuacao > melhor_pontuacao:
                melhor_pontuacao = pontuacao
                melhor_indice_caracteristicas = indices

        # Evaporação
        feromonio *= (1 - taxa_evaporacao)
        # Reforço
        for solucao, pontuacao in zip(solucoes_iteracao, pontuacoes_iteracao):
            for i in solucao:
                feromonio[i] += pontuacao

    return melhor_indice_caracteristicas, melhor_pontuacao

# ----------------------------
# PREPARAÇÃO DOS DADOS E EXECUÇÃO
# ----------------------------
colunas_entrada = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre',
                   'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida']
coluna_alvo = 'tem_sepse'

dados_entrada = df[colunas_entrada].copy()
dados_entrada = dados_entrada.apply(pd.to_numeric, errors='coerce')
dados_entrada.fillna(dados_entrada.mean(), inplace=True)

X = dados_entrada.values.astype(float)
y = df[coluna_alvo].values
if y.dtype == 'object' or str(y.dtype).startswith("category"):
    y = LabelEncoder().fit_transform(y)

melhores_indices, melhor_pontuacao = selecao_por_colonia_de_formigas(X, y)
melhores_nomes = [colunas_entrada[i] for i in melhores_indices]

print("\nMelhores características selecionadas:", melhores_nomes)
print("Melhor AUC (curva ROC):", melhor_pontuacao)


Iteração 1/15
Iteração 2/15
Iteração 3/15
Iteração 4/15
Iteração 5/15
Iteração 6/15
Iteração 7/15
Iteração 8/15
Iteração 9/15
Iteração 10/15
Iteração 11/15
Iteração 12/15
Iteração 13/15
Iteração 14/15
Iteração 15/15

Melhores características selecionadas: ['ida', 'fr', 'pam', 'pas', 'fc', 'uri', 'ph', 'leu']
Melhor AUC (curva ROC): 0.621087430042337


#### Colocando o beta maior para explorar mais locais

In [ ]:
def selecao_por_colonia_de_formigas(
    X, y,
    n_formigas=20,
    n_iteracoes=15,
    alfa=1,
    beta=2,
    taxa_evaporacao=0.3,
    min_caracteristicas_por_formiga=5,
    max_caracteristicas_por_formiga=8
):
    n_caracteristicas = X.shape[1]
    feromonio = np.ones(n_caracteristicas)

    # Heurística baseada na AUC ROC de cada feature individual
    heuristica = np.zeros(n_caracteristicas)
    for i in range(n_caracteristicas):
        try:
            heuristica[i] = roc_auc_score(y, X[:, i])
        except:
            heuristica[i] = 0.5  # AUC neutra para evitar erros
    heuristica += 1e-6  # evitar zeros

    def avaliar(indices):
        if len(indices) == 0:
            return 0
        X_sel = X[:, indices]
        modelo = LogisticRegression(solver='liblinear', class_weight='balanced', max_iter=1000)
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scores = cross_val_score(modelo, X_sel, y, cv=kfold, scoring='roc_auc')
        return scores.mean()

    melhor_indice_caracteristicas = []
    melhor_pontuacao = -np.inf

    for iteracao in range(n_iteracoes):
        solucoes_iteracao = []
        pontuacoes_iteracao = []

        print(f"Iteração {iteracao + 1}/{n_iteracoes}")

        for _ in range(n_formigas):
            # Probabilidades combinando feromônio e heurística (AUC)
            probabilidades = (feromonio ** alfa) * (heuristica ** beta)
            probabilidades /= probabilidades.sum()

            tamanho = np.random.randint(
                min_caracteristicas_por_formiga,
                min(max_caracteristicas_por_formiga, n_caracteristicas) + 1
            )

            indices = np.random.choice(
                range(n_caracteristicas),
                size=tamanho,
                replace=False,
                p=probabilidades
            )

            pontuacao = avaliar(indices)
            solucoes_iteracao.append(indices)
            pontuacoes_iteracao.append(pontuacao)

            if pontuacao > melhor_pontuacao:
                melhor_pontuacao = pontuacao
                melhor_indice_caracteristicas = indices

        # Evaporação
        feromonio *= (1 - taxa_evaporacao)
        # Reforço
        for solucao, pontuacao in zip(solucoes_iteracao, pontuacoes_iteracao):
            for i in solucao:
                feromonio[i] += pontuacao

    return melhor_indice_caracteristicas, melhor_pontuacao

# ----------------------------
# PREPARAÇÃO DOS DADOS E EXECUÇÃO
# ----------------------------
colunas_entrada = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre',
                   'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida']
coluna_alvo = 'tem_sepse'

dados_entrada = df[colunas_entrada].copy()
dados_entrada = dados_entrada.apply(pd.to_numeric, errors='coerce')
dados_entrada.fillna(dados_entrada.mean(), inplace=True)

X = dados_entrada.values.astype(float)
y = df[coluna_alvo].values
if y.dtype == 'object' or str(y.dtype).startswith("category"):
    y = LabelEncoder().fit_transform(y)

melhores_indices, melhor_pontuacao = selecao_por_colonia_de_formigas(X, y)
melhores_nomes = [colunas_entrada[i] for i in melhores_indices]

print("\nMelhores características selecionadas:", melhores_nomes)
print("Melhor AUC (curva ROC):", melhor_pontuacao)


### Aumentando a taxa de evaporação do algoritmo , mas deixando até 10 características 

In [5]:
def selecao_por_colonia_de_formigas(
    X, y,
    n_formigas=20,
    n_iteracoes=20,
    alfa=1.5,
    beta=2,
    taxa_evaporacao=0.3,
    min_caracteristicas_por_formiga=5,
    max_caracteristicas_por_formiga=10
):
    n_caracteristicas = X.shape[1]
    feromonio = np.ones(n_caracteristicas)

    # Heurística baseada em Mutual Information
    heuristica = mutual_info_classif(X, y, discrete_features='auto')
    heuristica = (heuristica - heuristica.min()) / (heuristica.max() - heuristica.min() + 1e-8)
    heuristica += 1e-6

    def avaliar(indices):
        if len(indices) == 0:
            return 0
        X_sel = X[:, indices]
        modelo = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
        kfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
        scores = cross_val_score(modelo, X_sel, y, cv=kfold, scoring='roc_auc')
        return scores.mean()

    melhor_indice_caracteristicas = []
    melhor_pontuacao = -np.inf

    for iteracao in range(n_iteracoes):
        solucoes_iteracao = []
        pontuacoes_iteracao = []

        print(f"Iteração {iteracao + 1}/{n_iteracoes}")

        for _ in range(n_formigas):
            probabilidades = (feromonio ** alfa) * (heuristica ** beta)
            probabilidades /= probabilidades.sum()

            tamanho = np.random.randint(
                min_caracteristicas_por_formiga,
                min(max_caracteristicas_por_formiga, n_caracteristicas) + 1
            )

            indices = np.random.choice(
                range(n_caracteristicas),
                size=tamanho,
                replace=False,
                p=probabilidades
            )

            pontuacao = avaliar(indices)
            solucoes_iteracao.append(indices)
            pontuacoes_iteracao.append(pontuacao)

            if pontuacao > melhor_pontuacao:
                melhor_pontuacao = pontuacao
                melhor_indice_caracteristicas = indices

        feromonio *= (1 - taxa_evaporacao)
        for solucao, pontuacao in zip(solucoes_iteracao, pontuacoes_iteracao):
            for i in solucao:
                feromonio[i] += pontuacao

    return melhor_indice_caracteristicas, melhor_pontuacao


# ----------------------------
# PREPARAÇÃO DOS DADOS E EXECUÇÃO
# ----------------------------
colunas_entrada = ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'cre',
                   'lac', 'leu', 'pla', 'ph', 'uri', 'hem', 'ida']
coluna_alvo = 'tem_sepse'

dados_entrada = df[colunas_entrada].copy()
dados_entrada = dados_entrada.apply(pd.to_numeric, errors='coerce')
dados_entrada.fillna(dados_entrada.mean(), inplace=True)

X = dados_entrada.values.astype(float)
y = df[coluna_alvo].values
if y.dtype == 'object' or str(y.dtype).startswith("category"):
    y = LabelEncoder().fit_transform(y)

# ESCALONAMENTO (opcional para RF, útil se trocar modelo no futuro)
scaler = StandardScaler()
X = scaler.fit_transform(X)

melhores_indices, melhor_pontuacao = selecao_por_colonia_de_formigas(X, y)
melhores_nomes = [colunas_entrada[i] for i in melhores_indices]

print("\nMelhores características selecionadas:", melhores_nomes)
print("Melhor AUC (curva ROC):", melhor_pontuacao)


KeyboardInterrupt: 